In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc # v1.6",
import sys
import matplotlib.pyplot as plt
import os.path
import anndata
from matplotlib import rcParams
import seaborn as sns
import numba
import mnnpy
import scipy

In [ ]:
#import full COVID-19 PBMC dataset
os.chdir('/home/ngr18/covid/')
covid_total = sc.read_h5ad('covid.h5ad')

In [ ]:
#Import BAL data GSE145926 (reannotated for DC subsets)

os.chdir('/home/ngr18/covid/external_dataset/BAL')
bal = sc.read_h5ad('full_bal.h5ad')

In [ ]:
#Subset PBMC data to myeloid populations and reorder categories (for dotplot visualisations)

blood_myeloid = covid_total[covid_total.obs.full_clustering.isin(['CD83_CD14_mono', 'CD14_mono', 
                                                                  'CD16_mono', 'C1_CD16_mono',
                                                                 'DC1', 'DC2', 'DC3', 'ASDC', 'pDC', 'DC_prolif',
                                                                 'Mono_prolif']),:]

blood_myeloid.obs.full_clustering = blood_myeloid.obs.full_clustering.cat.reorder_categories([
'DC1', 'DC2', 'DC3', 'ASDC', 'pDC', 'DC_prolif', 
    'CD83_CD14_mono', 'CD14_mono', 'CD16_mono', 'C1_CD16_mono', 'Mono_prolif'])

In [ ]:
#Concatenate PBMC and BAL data for visualisations

myeloid = anndata.concat([bal_myeloid, blood_myeloid], index_unique = None)

In [ ]:
##myeloid figure dotplot - fig 2A (left - RNA)

sc.pl.DotPlot(myeloid, [
'CLEC9A', 'CADM1', 'CLEC10A','CD1C', 'CD14', 'VCAN',
    'CCR7', 'LAMP3',
                            'AXL', 'SIGLEC6',
                            'LILRA4', 'ITM2C', 'GZMB',
    'IL1B', 'IER3', 'LDLR', 'CD83',
                         'S100A12', 'CSF3R', 
                        'FCGR3A', 'MS4A7', 'LILRB1', 'CSF1R', 'CDKN1C',
                         'C1QA', 'C1QB', 'C1QC', 'CCR1',
    
    'MARCO', 'MKI67', 'TOP2A'], 
              log = True, 
              groupby='full_clustering').style(cmap='Blues',dot_edge_color='black', dot_edge_lw=1).swap_axes(False).show(True)


In [ ]:
##myeloid figure dotplot - fig 2A (right - protein) - using only PBMC data (no CITEseq data for BAL)

sc.pl.DotPlot(blood_myeloid, ['AB_CD141', 'AB_CLEC9A', 'AB_KIT','AB_BTLA',
                            'AB_CD1C', 'AB_CD101', 'AB_FcERIa',
                            'AB_CD5', 
                            'AB_CD123', 'AB_CD45RA', 'AB_CD304',
                            'AB_CD14', 'AB_CD99', 'AB_CD64',  
                            'AB_CR1', 'AB_ITGAM',
                            'AB_CD16', 'AB_C5AR1',
                              'AB_CX3CR1'], 
              log = True, 
              groupby='full_clustering',  expression_cutoff=0.15).style(cmap='YlOrRd',dot_edge_color='black', dot_edge_lw=1).swap_axes(False).show(True)


In [ ]:
#subset data again to blood monocyte and BAL macrophages

monos = myeloid[myeloid.obs.full_clustering.isin(['CD83_CD14_mono', 'CD14_mono', 'CD16_mono', 'C1_CD16_mono', 
                                                 'bal_Mac']),:]

In [ ]:
#Figure 2C left/upper - analysis of healthy only
healthy = monos[monos.obs.Status_on_day_collection_summary.isin(['bal_healthy', 'Healthy']),:]

sc.pp.normalize_total(healthy, target_sum=1e4)

sc.pp.log1p(healthy)

sc.pp.highly_variable_genes(healthy, n_top_genes = 3000, flavor = 'seurat')

healthy.raw = healthy

healthy = healthy[:, healthy.var.highly_variable]

sc.pp.scale(healthy, max_value=10)

sc.tl.pca(healthy, svd_solver='arpack')

sc.external.pp.harmony_integrate(healthy, 'sample_id', basis='X_pca', adjusted_basis='X_pca_harmony')

sc.pp.neighbors(healthy, n_neighbors=10, n_pcs=50, use_rep = 'X_pca_harmony')

sc.tl.paga(healthy, groups='full_clustering')

#recolor clusters for consistency with bar graph

healthy.uns['full_clustering_colors'][0] = '#76DDC9'
healthy.uns['full_clustering_colors'][1] = '#E28686'
healthy.uns['full_clustering_colors'][2] = '#C2A3E2'
healthy.uns['full_clustering_colors'][3] = '#991111'
healthy.uns['full_clustering_colors'][4] = '#ffff00'


sc.pl.paga(test, color=['full_clustering'], threshold = 0.11, save = 'healthy_myeloid_PAGA.pdf',
           fontsize = 0, node_size_scale = 10, min_edge_width = 5, frameon = False)

In [ ]:
#Figure 2C left/lower - limit to only covid samples
covid_mono = covid_mono[covid_mono.obs.Status_on_day_collection_summary.isin(['Asymptomatic', 'Critical', 
                                                            'Mild', 'Moderate', 'Severe',
                                                            'bal_mild', 'bal_severe']),:]

sc.pp.normalize_total(covid_mono, target_sum=1e4)

sc.pp.log1p(covid_mono)

sc.pp.highly_variable_genes(covid_mono, n_top_genes = 3000, flavor = 'seurat')

covid_mono.raw = covid_mono

covid_mono = covid_mono[:, covid_mono.var.highly_variable]

sc.pp.scale(covid_mono, max_value=10)

sc.tl.pca(covid_mono, svd_solver='arpack')

sc.external.pp.harmony_integrate(covid_mono, 'sample_id', basis='X_pca', adjusted_basis='X_pca_harmony')

sc.pp.neighbors(covid_mono, n_neighbors=10, n_pcs=50, use_rep = 'X_pca_harmony')

sc.tl.paga(covid_mono, groups='full_clustering')

#Again, recolor for consistency with bar graph

covid_mono.uns['full_clustering_colors'][0] = '#76DDC9'
covid_mono.uns['full_clustering_colors'][1] = '#E28686'
covid_mono.uns['full_clustering_colors'][2] = '#C2A3E2'
covid_mono.uns['full_clustering_colors'][3] = '#991111'
covid_mono.uns['full_clustering_colors'][4] = '#ffff00'


sc.pl.paga(covid_mono, color=['full_clustering'], threshold = 0.2, save = 'covid_myeloid_PAGA.pdf',
           fontsize = 0, node_size_scale = 10, min_edge_width = 5, frameon = False)

In [ ]:
#Pseudotime plot

covid_mono.uns['iroot'] = np.flatnonzero(covid_mono.obs['full_clustering']  == 'CD83_CD14_mono')[0]


sc.pp.log1p(covid_mono)
sc.pp.scale(covid_mono)
           
           gene_names = ['NFKBIA',  'KLF6', 'VIM', 'CD14', 'S100A8',
    'HLA-DPA1', 'HLA-DPB1', 'FCGR3A','CTSC', 'CTSL', 
    'CCR1','RSAD2', 'CCL2', 'CXCL10', 'CCL7', 'TNFSF10']                              # monocyte

paths = [('erythrocytes', ['mono1', 'mono2', 'mono3', 'mono4', 'bal_Mac']),
             ('neutrophils', ['mono1', 'mono2', 'mono3', 'mono4', 'bal_Mac']),
             ('monocytes', ['mono1', 'mono2', 'mono3', 'mono4', 'bal_Mac'])]

test.obs['distance'] = test.obs['dpt_pseudotime']

_, axs = plt.subplots(ncols=3, figsize=(6, 5), gridspec_kw={'wspace': 0.05, 'left': 0.12})
plt.subplots_adjust(left=0.05, right=0.98, top=0.82, bottom=0.2)
for ipath, (descr, path) in enumerate(paths):
    _, data = sc.pl.paga_path(
            test, path, gene_names,
            show_node_names=False,
            ax=axs[ipath],
            ytick_fontsize=12,
            left_margin=0.15,
            n_avg=50,
            annotations=['distance'],
            show_yticks=True if ipath==0 else False,
            show_colorbar=False,
            color_map='coolwarm',
            color_maps_annotations={'distance': 'viridis'},
            title='{} path'.format(descr),
            return_data=True,
            show=False, normalize_to_zero_one = True)